In [1]:
# 모듈 임포트
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from split_data import train_test_split_timeseries, prepare_rl_data, validate_rl_data
from offline_RLs import (
    BehaviorCloning, ConservativeQL, ImplicitQL, TD3BC, OfflineDDPG,
    compare_algorithms, create_replay_buffer_from_data, normalize_data
)
import time


Using device: cpu


In [2]:
# 데이터 불러오기
path_root = os.path.dirname(os.path.abspath('.'))
# data = pd.read_csv(os.path.join(path_root, 'data', 'data0904.csv'))
data = pd.read_csv(os.path.join('data0904.csv'))

print("=" * 60)
print("Offline RL Algorithms Comparison Test")
print("=" * 60)

# 변수 정의
action_tag = 'DGAN Compressor 질소가스 유량'
target_tag = '가스터빈 후단 질소산화물 농도'

# 1. Train/Test 분할
print("\n1. Data Splitting...")
train_data, test_data, split_info = train_test_split_timeseries(data, test_ratio=0.2)

# 2. 강화학습 데이터 준비 (Train)
print("\n2. Preparing RL Data...")
train_rl_data = prepare_rl_data(train_data, action_tag, target_tag)

# 3. 데이터 검증
print("\n3. Data Validation...")
is_valid, validation_info = validate_rl_data(train_rl_data)

if not is_valid:
    print("❌ Data validation failed!")
    print(validation_info)
    exit()

Offline RL Algorithms Comparison Test

1. Data Splitting...
Data split completed:
  Total samples: 120960
  Train samples: 96768 (80.0%)
  Test samples: 24192 (20.0%)
  Train period: 2025-02-17 00:00:00 ~ 2025-02-22 14:23:55
  Test period: 2025-02-22 14:24:00 ~ 2025-02-23 23:59:55

2. Preparing RL Data...
RL data preparation completed:
  Samples: 96767
  State dim: 11
  Action dim: 1
  Target variable: 가스터빈 후단 질소산화물 농도 (index: 10)
  Action bounds: [26.10, 47.19]
  Average reward: -5.000
  Reward std: 0.207

3. Data Validation...
✓ RL data validation passed successfully


In [4]:
print(train_rl_data['states'].shape)
print(train_rl_data['actions'].shape)
print(train_rl_data['rewards'].shape)
print(train_rl_data['next_states'].shape)
print(train_rl_data['done'].shape)

(96767, 11)
(96767, 1)
(96767,)
(96767, 11)
(96767,)


In [5]:
name = 'BC'
algorithm_class = BehaviorCloning
iterations = 5000

print("Comparing multiple offline RL algorithms...")
print("=" * 60)

# 데이터 정규화
normalized_data, state_scaler, action_scaler = normalize_data(train_rl_data)

# 공통 파라미터
state_dim = normalized_data['states'].shape[1]
action_dim = normalized_data['actions'].shape[1]
max_action = 1.0  # 정규화 후 대략적인 최대값

# Replay buffer 생성
replay_buffer = create_replay_buffer_from_data(normalized_data)

print(f"\n🚀 Training {name}...")
print("-" * 40)

# 알고리즘 인스턴스 생성
if name == 'BC':
    algorithm = algorithm_class(state_dim, action_dim, max_action)
    training_results = algorithm.train(replay_buffer, iterations=iterations//5)  # BC는 더 적게
else:
    algorithm = algorithm_class(state_dim, action_dim, max_action)
    training_results = algorithm.train(replay_buffer, iterations=iterations)


results = {
    'algorithm': algorithm,
    'training_results': training_results,
    'state_scaler': state_scaler,
    'action_scaler': action_scaler
        }

print(f"✅ {name} training completed!")

Comparing multiple offline RL algorithms...
Replay buffer created with 96767 samples

🚀 Training BC...
----------------------------------------
Training Behavior Cloning...
BC Iteration 200/1000, Loss: 0.176902
BC Iteration 400/1000, Loss: 0.239938
BC Iteration 600/1000, Loss: 0.173163
BC Iteration 800/1000, Loss: 0.165050
BC Iteration 1000/1000, Loss: 0.188534
✅ BC training completed!


In [6]:
algorithm = results['algorithm']
training_results = results['training_results']
state_scaler = results['state_scaler']
action_scaler = results['action_scaler']

In [7]:
print(len(training_results['losses']))

1000


In [8]:
# 4. 알고리즘 정의 및 비교
print("\n4. Running Algorithm Comparison...")

algorithms = {
    'BC': BehaviorCloning,
    'CQL': ConservativeQL, 
    'IQL': ImplicitQL,
    'TD3+BC': TD3BC,
    'DDPG': OfflineDDPG
}

# 학습 시작
start_time = time.time()
results = compare_algorithms(algorithms, train_rl_data, iterations=2000)  # 테스트용으로 줄임
end_time = time.time()

print(f"\n⏱️ Total training time: {end_time - start_time:.2f} seconds")


4. Running Algorithm Comparison...
Comparing multiple offline RL algorithms...
Replay buffer created with 96767 samples

🚀 Training BC...
----------------------------------------
Training Behavior Cloning...
BC Iteration 200/400, Loss: 0.146544
BC Iteration 400/400, Loss: 0.175878
✅ BC training completed!

🚀 Training CQL...
----------------------------------------
Training Conservative Q-Learning...
CQL Iteration 1000/2000
  Critic Loss: 1.844166
  CQL Loss: -2.379364
  Actor Loss: 14.587351
CQL Iteration 2000/2000
  Critic Loss: 22.379984
  CQL Loss: -6.420288
  Actor Loss: 44.663806
✅ CQL training completed!

🚀 Training IQL...
----------------------------------------
Training Implicit Q-Learning...
IQL Iteration 1000/2000
  Critic Loss: 243.228867
  Value Loss: 5.456341
  Actor Loss: -6.241105
IQL Iteration 2000/2000
  Critic Loss: 258.481628
  Value Loss: 4.265056
  Actor Loss: -4.678623
✅ IQL training completed!

🚀 Training TD3+BC...
----------------------------------------
Traini

In [ ]:
# 5. 결과 분석 및 시각화
print("\n5. Results Analysis...")

# 학습 곡선 시각화
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Offline RL Algorithms Training Progress', fontsize=16)

for i, (name, result) in enumerate(results.items()):
    row = i // 3
    col = i % 3
    
    training_results = result['training_results']
    
    if 'losses' in training_results:  # BC의 경우
        axes[row, col].plot(training_results['losses'], label='Loss')
        axes[row, col].set_title(f'{name} - Loss')
    elif 'actor_losses' in training_results:  # 나머지 알고리즘들
        if training_results['actor_losses']:
            axes[row, col].plot(training_results['actor_losses'], label='Actor Loss', alpha=0.7)
        if 'critic_losses' in training_results:
            # critic loss는 더 자주 업데이트되므로 간격을 맞춤
            critic_x = np.arange(len(training_results['critic_losses']))
            axes[row, col].plot(critic_x, training_results['critic_losses'], 
                              label='Critic Loss', alpha=0.7)
        axes[row, col].set_title(f'{name} - Training Losses')
    
    axes[row, col].set_xlabel('Iterations')
    axes[row, col].set_ylabel('Loss')
    axes[row, col].legend()
    axes[row, col].grid(True, alpha=0.3)

# 빈 subplot 제거
if len(results) < 6:
    axes[1, 2].remove()

plt.tight_layout()
plt.show()

In [ ]:
# 6. 간단한 성능 테스트 (몇 개 샘플로)
print("\n6. Simple Performance Test...")

# Test 데이터도 RL 형태로 변환
test_rl_data = prepare_rl_data(test_data, action_tag, target_tag)
print("=" * 40)

# 정규화된 테스트 데이터 준비
test_normalized_data, _, _ = normalize_data(test_rl_data)

# 각 알고리즘의 성능 테스트
performance_results = {}

for name, result in results.items():
    print(f"\nTesting {name}...")
    
    algorithm = result['algorithm']
    state_scaler = result['state_scaler']
    action_scaler = result['action_scaler']
    
    # 테스트 샘플 몇 개로 성능 확인
    n_test_samples = min(100, len(test_rl_data['states']))
    test_states = test_rl_data['states'][:n_test_samples]
    test_actions = test_rl_data['actions'][:n_test_samples]
    
    # 정규화
    test_states_norm = state_scaler.transform(test_states)
    test_actions_norm = action_scaler.transform(test_actions)
    
    # 예측 액션
    predicted_actions_norm = []
    for state in test_states_norm:
        pred_action = algorithm.select_action(state)
        predicted_actions_norm.append(pred_action)
    
    predicted_actions_norm = np.array(predicted_actions_norm)
    
    # 원래 스케일로 변환
    predicted_actions = action_scaler.inverse_transform(predicted_actions_norm)
    
    # MSE 계산
    mse = np.mean((predicted_actions.flatten() - test_actions[:n_test_samples].flatten()) ** 2)
    mae = np.mean(np.abs(predicted_actions.flatten() - test_actions[:n_test_samples].flatten()))
    
    performance_results[name] = {
        'mse': mse,
        'mae': mae
    }
    
    print(f"  MSE: {mse:.4f}")
    print(f"  MAE: {mae:.4f}")

In [ ]:
# 7. 최종 결과 요약
print("\n" + "=" * 60)
print("FINAL RESULTS SUMMARY")
print("=" * 60)

print(f"{'Algorithm':<10} {'MSE':<10} {'MAE':<10} {'Status':<15}")
print("-" * 50)

best_mse_algo = min(performance_results.items(), key=lambda x: x[1]['mse'])
best_mae_algo = min(performance_results.items(), key=lambda x: x[1]['mae'])

for name, perf in performance_results.items():
    status = ""
    if name == best_mse_algo[0]:
        status += "🏆MSE "
    if name == best_mae_algo[0]:
        status += "🏆MAE"
    
    print(f"{name:<10} {perf['mse']:<10.4f} {perf['mae']:<10.4f} {status:<15}")

print(f"\nBest MSE: {best_mse_algo[0]} ({best_mse_algo[1]['mse']:.4f})")
print(f"Best MAE: {best_mae_algo[0]} ({best_mae_algo[1]['mae']:.4f})")

# 8. 액션 예측 비교 시각화
print("\n8. Action Prediction Visualization...")

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Action Predictions vs Ground Truth (First 50 samples)', fontsize=16)

n_viz_samples = min(50, len(test_rl_data['states']))

for i, (name, result) in enumerate(results.items()):
    row = i // 3
    col = i % 3
    
    algorithm = result['algorithm']
    state_scaler = result['state_scaler'] 
    action_scaler = result['action_scaler']
    
    # 예측
    test_states_norm = state_scaler.transform(test_rl_data['states'][:n_viz_samples])
    predicted_actions_norm = []
    
    for state in test_states_norm:
        pred_action = algorithm.select_action(state)
        predicted_actions_norm.append(pred_action)
    
    predicted_actions = action_scaler.inverse_transform(np.array(predicted_actions_norm))
    actual_actions = test_rl_data['actions'][:n_viz_samples]
    
    # 플롯
    x_axis = np.arange(n_viz_samples)
    axes[row, col].plot(x_axis, actual_actions.flatten(), 'b-', label='Actual', linewidth=2)
    axes[row, col].plot(x_axis, predicted_actions.flatten(), 'r--', label='Predicted', linewidth=2, alpha=0.7)
    axes[row, col].set_title(f'{name}')
    axes[row, col].set_xlabel('Sample')
    axes[row, col].set_ylabel('Action Value')
    axes[row, col].legend()
    axes[row, col].grid(True, alpha=0.3)

# 빈 subplot 제거
if len(results) < 6:
    axes[1, 2].remove()

plt.tight_layout()
plt.show()

print("\n🎉 All tests completed successfully!")
print("=" * 60)